#### Import standard Python modules

In [1]:
import datetime
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.externals import joblib

#### Import Azure ML SDK modules

In [2]:
import azureml.core
from azureml.core import Workspace
from azureml.core.model import Model
from azureml.core import Experiment
from azureml.core.webservice import Webservice
from azureml.core.image import ContainerImage
from azureml.core.webservice import AciWebservice
from azureml.core.conda_dependencies import CondaDependencies

#### Check Azure ML SDK version

In [3]:
print(azureml.core.VERSION)

1.0.57


#### Create Azure ML Workspace )or get Workspace Info)

In [4]:
#### following code can be used when there is not such ML workspace existing

#AZ_SUBSCRIPTION_ID='7b03fb67-8d44-40ef-9326-c8955518fc75'
#ws = Workspace.create(name='rw-ml01-aamls',
#                      subscription_id=AZ_SUBSCRIPTION_ID, 
#                      resource_group='CE-DEV-ROGERLAB-RG',
#                      create_resource_group=True,
#                      location='westeurope'
#                    )
#print('\ncreation completed')

In [4]:
#### from azureml.core import Workspace if already existing
#### load workspace configuration from the config.json file in the current folder.

ws = Workspace.from_config()
print(ws.name, ws.location, ws.resource_group, ws.location, sep='\t')

rw-ml01-aamls	westeurope	CE-DEV-ROGERLAB-RG	westeurope


In [7]:
#!cat config.json

#### Write configuration to local file

In [7]:
ws.write_config()
print('write_config completed')

write_config completed


#### Create Azure ML Experiment

In [8]:
exp = Experiment(workspace=ws, name='salexp-pd')

#### Start logging metrics

In [9]:
run = exp.start_logging()                   
run.log("Experiment start time", str(datetime.datetime.now()))

#### Load salary dataset

In [10]:
sal = pd.read_csv('data/sal.csv',header=0, index_col=None)
X = sal[['x']] # change from pd series to array!
y = sal['y']

print(sal.head(), '\n')
print(X[:5], '\n')
print(y[:5])

   x       y
0  0  103100
1  1  104900
2  2  106800
3  3  108700
4  4  110400 

   x
0  0
1  1
2  2
3  3
4  4 

0    103100
1    104900
2    106800
3    108700
4    110400
Name: y, dtype: int64


#### Split the train and test data

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=10)

#### Train the model

In [12]:
lm = LinearRegression() 
lm.fit(X_train,y_train) 

print('training score: ', lm.score(X_train,y_train))
print('testing score: ', lm.score(X_test,y_test))

training score:  0.9999818159227116
testing score:  0.9999698027222448


#### Freeze the model & create a .pkl file being as a model

In [13]:
# here is the key part - to generate a model from traing classifier!!

filename = 'outputs/sal_model.pkl' # give it a name, and best practice is to have an folder named 'outputs/'
joblib.dump(lm, filename) # classifer turns to a .pkl file!

['outputs/sal_model.pkl']

In [14]:
###alternative way to create the .pkl file
###import pickle
###pickle.dump(lm, open('outputs/sal_model.pkl','wb'))
###loaded_model = pickle.load(open('outputs/sal_model.pkl','rb'))

#### Test the model by calling the .pkl file

In [14]:
filename = 'outputs/sal_model.pkl'
loaded_model = joblib.load(filename)

# model.predict - use it like a function,
y=loaded_model.predict([[30]])[0]   # note: the double [[]], as input will need to be an array! 

print(y)

158322.15447154472


#### Log metrics to Azure ML Experiment

In [15]:
run.log('Intercept :', lm.intercept_)
run.log('Slope :', lm.coef_[0])

#### End Azure ML Experiment

In [16]:
run.log("Experiment end time", str(datetime.datetime.now()))
run.complete()

#### Get Portal URL

In [17]:
print(run.get_portal_url())

https://mlworkspace.azure.ai/portal/subscriptions/7b03fb67-8d44-40ef-9326-c8955518fc75/resourceGroups/CE-DEV-ROGERLAB-RG/providers/Microsoft.MachineLearningServices/workspaces/rw-ml01-aamls/experiments/salexp-pd/runs/3b720973-fe7e-4363-abd9-83ba84ca10db


#### Register the model

In [18]:
model = Model.register(model_path = "outputs/sal_model.pkl",
                       model_name = "sal_model",
                       tags = {"key": "1"},
                       description = "Salary Prediction",
                       workspace = ws)

Registering model sal_model


In [19]:
## download the model to hav a look
model.download(target_dir='downloaded_model/', exist_ok = True) # folder 'downloaded_model' is being generated here

'downloaded_model\\sal_model.pkl'

In [20]:
p = joblib.load('outputs/sal_model.pkl')
p.predict([[20],[30],[40]])

array([139920.39295393, 158322.15447154, 176723.91598916])

#### Define Azure ML Deploymemt configuration

In [22]:
aciconfig

#### Create enviroment configuration file

In [33]:
# Target is to generate an .yml file - package enviornment
salenv = CondaDependencies()
salenv.add_conda_package("scikit-learn")
salenv.add_conda_package("inference-schema")

with open("salenv.yml","w") as f:
    f.write(salenv.serialize_to_string())
with open("salenv.yml","r") as f:
    print(f.read())

# Conda environment specification. The dependencies defined in this file will

# be automatically provisioned for runs with userManagedDependencies=False.


# Details about the Conda environment file format:

# https://conda.io/docs/user-guide/tasks/manage-environments.html#create-env-file-manually


name: project_environment
dependencies:
  # The python interpreter version.

  # Currently Azure ML only supports 3.5.2 and later.

- python=3.6.2

- pip:
    # Required packages for AzureML execution, history, and data preparation.

  - azureml-defaults

- scikit-learn
- inference-schema
channels:
- conda-forge



#### Create Azure ML Scoring file

In [32]:
%%writefile score.py

import json
import numpy as np
import os
import pickle
from sklearn.externals import joblib
from sklearn.linear_model import LogisticRegression

from azureml.core.model import Model

#from inference_schema.schema_decorators import input_schema, output_schema
#from inference_schema.parameter_types.numpy_parameter_type import NumpyParameterType
#from inference_schema.parameter_types.pandas_parameter_type import PandasParameterType

def init():
    global model
    # retrieve the path to the model file using the model name
    model_path = Model.get_model_path('sal_model')
    model = joblib.load(model_path)


#input_sample = pd.DataFrame(data=[{
    # This is a decimal type sample. Use the data type that reflects this column in your data
#    "age": 28,
    # This is a string type sample. Use the data type that reflects this column in your data
    #"input_name_2": "value2",
    # This is a integer type sample. Use the data type that reflects this column in your data
   # "input_name_3": 3
#}])

# This is a integer type sample. Use the data type that reflects the expected result
#output_sample = np.array([0])


#@input_schema('data', PandasParameterType(input_sample))
#@output_schema(NumpyParameterType(output_sample))

def run(raw_data):
    data = np.array(json.loads(raw_data)['data'])
    # make prediction
    y_hat = model.predict(data)
    return json.dumps(y_hat.tolist())

Overwriting score.py


#### Deploy the model to Azure Container Instance


In [25]:
### this orginal script does not work

#%%time
#image_config = ContainerImage.image_configuration(execution_script="score.py", 
                                                  runtime="python", 
                                                  conda_file="salenv.yml")
#print('image_config completed!')
#image_config

image_config completed!
Wall time: 3.99 ms


In [34]:
aciconfig = AciWebservice.deploy_configuration(cpu_cores=1, 
                                               memory_gb=1, 
                                               tags={"data": "Salary",  "method" : "sklearn"}, 
                                               description='Predict Stackoverflow Salary')

In [39]:
%%time

from azureml.core.webservice import Webservice
from azureml.core.model import InferenceConfig

inference_config = InferenceConfig(runtime= "python", 
                                   entry_script="score.py",
                                   conda_file="salenv.yml")

service = Model.deploy(workspace=ws, 
                       name='sal-predic-svc', 
                       models=[model], 
                       inference_config=inference_config, 
                       deployment_config=aciconfig)

service.wait_for_deployment(show_output=True)

Creating service
Running.......................
SucceededACI service creation operation finished, operation "Succeeded"
Wall time: 2min 14s


#### Expose web service / Get the web serive info from Workspaces

In [5]:
#services = Webservice.list(ws)
#print(services[0].scoring_uri)
#print(services[0].swagger_uri)
#print('\n')

# FROM SERVICE NAME TO GET THE SERVICE OBJECT
service = Webservice(workspace=ws, name='sal-predic-svc')

print('the service delopyed uri is as following : ')
print(service.scoring_uri)
print(service.swagger_uri)

the service delopyed uri is as following : 
http://50353d27-0ed3-4a70-81df-24f786986a0b.westeurope.azurecontainer.io/score
http://50353d27-0ed3-4a70-81df-24f786986a0b.westeurope.azurecontainer.io/swagger.json


#### Get the Web Service URL

In [6]:
print(service.scoring_uri)

http://50353d27-0ed3-4a70-81df-24f786986a0b.westeurope.azurecontainer.io/score


In [11]:
!curl -X POST \
	-H 'Content-Type':'application/json' \
	-d '{"data":[[45]]}' \
	http://50353d27-0ed3-4a70-81df-24f786986a0b.westeurope.azurecontainer.io/score

{"message": "Expects Content-Type to be application/json"}


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100    73  100    58  100    15    109     28 --:--:-- --:--:-- --:--:--   137
100    73  100    58  100    15    109     28 --:--:-- --:--:-- --:--:--   137


### Consume the service by calling HTTP endpoint (REST API)

In [10]:
import requests
import numpy as np
import json

# send a random row from the test set to score

data = {"data":
        [
            [35],
            [40],
            [26]
        ]
        }

# Convert to JSON string
input_data = json.dumps(data)

#input_data = "{\"data\": [" + str(list(X_test[random_index])) + "]}"

headers = {'Content-Type':'application/json'}

# for AKS deployment you'd need to the service key in the header as well
# api_key = service.get_key()
# headers = {'Content-Type':'application/json',  'Authorization':('Bearer '+ api_key)} 

scoring_uri = service.scoring_uri
resp = requests.post(scoring_uri, input_data, headers=headers)


print("POST to url", scoring_uri)
print("input data:\n", input_data)

print("\nprediction out is as following:\n", resp.text)

print('\nresp output type as:', type(resp))


POST to url http://50353d27-0ed3-4a70-81df-24f786986a0b.westeurope.azurecontainer.io/score
input data:
 {"data": [[35], [40], [26]]}

prediction out is as following:
 "[167523.0352303523, 176723.9159891599, 150961.44986449866]"

resp output type as: <class 'requests.models.Response'>


#### Delete Workspace and clean up resources

In [29]:
# ws.delete()